In [ ]:
import os
import json

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain import hub
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict


In [ ]:
load_dotenv()
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = os.environ["LANGSMITH_API_KEY"]

In [ ]:
osdi25_path = "data/osdi_atc25.json"
def json_to_docs(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)

    docs = []
    for session in data:
        for paper in data[session]:
            if paper["abstract"] is None:
                continue
            try:
                doc = Document(
                    page_content=paper["abstract"],
                    metadata={
                        "title": paper["title"],
                        "authors": paper["authors"],
                        "link": paper["link"],
                        "session": session
                    }
                )
                docs.append(doc)
            except Exception as e:
                print(e)
                print(paper)

    
    return docs

docs = []
docs += json_to_docs("data/original/osdi_atc25.json")
# docs += json_to_docs("data/original/osdi24_sessions.json")
# docs += json_to_docs("data/original/nsdi25_sessions.json")
# docs += json_to_docs("data/original/sosp24_sessions.json")
# docs += json_to_docs("data/original/eurosys25_sessions.json")
        

In [ ]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.environ["OPENAI_API_KEY"]
)
vector_store = InMemoryVectorStore(embeddings)

In [ ]:
llm = ChatOpenAI(
    model="qwen/qwen3-30b-a3b:free",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"]
)

In [ ]:
vector_store.add_documents(docs)

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
response = graph.invoke({"question": "Point out a list of relevant papers for LLM inference"})
print(response["answer"])